<a href="https://colab.research.google.com/github/ffviana/NUTRECON/blob/main/Simulation/NUTRECON_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NUTRECON General Task Description
## Same-type trials
In same-type trials, subjects must choose between a small but guaranteed quantity of a reward (Reference option) vs a probability (13, 22, 38, 50 or 75%) of winning a fixed amount of the same reward or getting nothing (Lottery Option).

* Lottery probabilities: 13, 22, 38, 50 or 75%
* Quantities:
  * Money:
    * Reference option: 2 €
    * Lottery option: 2, 4.5, 10, 22.5 or 50 €

  * Yogurts A and B	
    * Reference option: 1
    * Lottery option: 1, 2, 5, 11 or 25 cups (containers?)

There are (nº of lottery probs * nº of lottery quantities) unique lottery (UL) options in per reward type. 

We can play around with the number of UL option repetitions. (6 in the paper)

_______
## Mixed-type trials:
In mixed-type trials, subjects must choose between a small but guaranteed amount of Money (Reference option) vs a probability (13, 22, 38, 50 or 75%) of winning a fixed amount of Yogurt A, Yogurt B, Water or getting nothing (Lottery Option).
Quantities:
Money
Reference: 0.20€

* Lottery probabilities: 13, 22, 38, 50 or 75%
* Quantities:
  * Reference option: 0.20 €
  * Lottery option: 1, 2, 3, 4 or 6 cups (of yogurt A or yoburt B)

  * Yogurts A and B	
    * Reference option: 1
    * Lottery option: 1, 2, 5, 11 or 25 cups (containers?)


## Framework

From Levy and Glimcher 2011, the estimated utility (EU) is given by:

\begin{align}
        EU(p_{i},X_{i},α_{i}) = p_{i} × X_{i}^{α_{i}}
\end{align}

where $p_{i}$ is the probability of recieving a quantity $X_{i}$ of reward $i$.  
<br>
From Levy and Glimcher 2011, with $EU_{L}$ and $EU_{R}$ as the EU for lottery and reference options, respcetively, the probability $P_{L}$ of choosing the lottery option is given by:

\begin{align}
        P_{L}(β_{i}) = \frac{1}{1 + e^{β_{i} * (EU_{L} - EU_{R})}}
\end{align}

however, if $EU_{L} > EU_{R} \rightarrow P_{L} < 0.5$. Thus, the correct equation is:
\begin{align}
        P_{L}(β_{i}) = 1 - \frac{1}{1 + e^{β_{i} * (EU_{L} - EU_{R})}}
\end{align}


In [54]:
# Import Packages
import numpy as np
import pandas as pd
from random import randint
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

## Same-type trials

### Step by step for 1 Subject

#### Define Task parameters


In [55]:
# Same-type task variables

uniqueLott_Nreps= 6                                 # Unique Lottery Repititions  
st_refPs = [1]
st_lottPs = [0.13, 0.22, 0.38, .50, .75]            # Same-type Trials Lottery probabilities

money_id = 'Money'
money_refQs = [2]                                   # Euros
money_lottQs = [2, 4.5, 10, 22.5, 50]               # Euros
money_refPars = [money_id, money_refQs, st_refPs]
money_lottPars = [money_id, money_lottQs, st_lottPs]

yogiCplus_id = 'C+'
yogiCplus_refQs = [1]                               # Cups of C+ yogurt 
yogiCplus_lottQs = [1, 2, 5, 11, 25]                # Cups of C+ yogurt
yogiCplus_refPars = [yogiCplus_id, yogiCplus_refQs, st_refPs]
yogiCplus_lottPars = [yogiCplus_id, yogiCplus_lottQs, st_lottPs]

yogiCminus_id = 'C-'
yogiCminus_refQs = yogiCplus_refQs                  # Cups of C- yogurt 
yogiCminus_lottQs = yogiCplus_lottQs                # Cups of C- yogurt 
yogiCminus_refPars = [yogiCminus_id, yogiCminus_refQs, st_refPs]
yogiCminus_lottPars = [yogiCminus_id, yogiCminus_lottQs, st_lottPs]



#### Define subject-specific parameters

In [56]:
# %% Subject specific parameters
money_alpha = 0.7
yogiCplus_alpha = 0.8
yogiCminus_alpha = 0.8

money_beta = 1.8
yogiCplus_beta = money_beta
yogiCminus_beta = money_beta

alphas = {'Money' : money_alpha,
             'C+' : yogiCplus_alpha,
             'C-' : yogiCminus_alpha}
betas = {'Money' : money_beta,
             'C+' : yogiCplus_beta,
             'C-' : yogiCminus_beta}

#### Simulate Choice Data

##### Get Trial Combinations + alphas and betas

In [57]:
column_names = ['ref_type', 'ref_qt', 'ref_prob' ,
                'lott_type', 'lott_qt', 'lott_prob',
                'ref_alpha', 'lott_alpha', 'beta',
                'ref_EU', 'lott_EU', 'pL', 'choice',
                'ref_alphaEstimates', 'lott_alphaEstimates', 'beta_estimates']

def get_uniqueCombinations(ref_pars, lott_pars):
  ref_uniqueCombs = [ [ref_pars[0],qt,prob] for qt in ref_pars[1] for prob in ref_pars[2] ]
  lott_uniqueCombs = [ [lott_pars[0],qt,prob] for qt in lott_pars[1] for prob in lott_pars[2] ]

  uniqueCombs = [ref + lott for ref in ref_uniqueCombs for lott in lott_uniqueCombs]
  return uniqueCombs

def get_allCombinations(ref_parsList, lott_parsList, N_uniqueReps):
  
  all_uniqueLottPars = []
  for p in range(len(ref_parsList)):
    uniqueCombs = get_uniqueCombinations(ref_parsList[p], lott_parsList[p])
    all_uniqueLottPars += uniqueCombs
  
  LotteryCombs = all_uniqueLottPars * N_uniqueReps
  return LotteryCombs

# Generate Unique Lotteries 
all_refPars = [money_refPars, yogiCplus_refPars, yogiCminus_refPars]
all_lottPars = [money_lottPars, yogiCplus_lottPars, yogiCminus_lottPars]
LotteryCombinations = get_allCombinations(all_refPars, all_lottPars, uniqueLott_Nreps)
 
#money_uniqueLottCombs = get_uniqueCombinations(money_refPars, money_lottPars)
#yogiCplus_uniqueLottCombs = get_uniqueCombinations(yogiCplus_refPars, yogiCplus_lottPars)
#yogiCminus_uniqueLottCombs = get_uniqueCombinations(yogiCminus_refPars, yogiCminus_lottPars)
#allUniqueLotteryCombinations = money_uniqueLottCombs + yogiCplus_uniqueLottCombs + yogiCminus_uniqueLottCombs
#LotteryCombinations = allUniqueLotteryCombinations * uniqueLott_Nreps         # Lottery combinations for all reward types

N_trials = len(LotteryCombinations)                 # Total number of Same-type trials

print('Total number of Same-type trials = {}'.format(N_trials))

sameType_sim_df = pd.DataFrame(LotteryCombinations, columns = column_names[:6])

# Get alpha and beta values
sameType_sim_df[column_names[6]] = sameType_sim_df[column_names[0]].replace(alphas)
sameType_sim_df[column_names[7]] = sameType_sim_df[column_names[3]].replace(alphas)
sameType_sim_df[column_names[8]] = sameType_sim_df[column_names[3]].replace(betas)
sameType_sim_df.sample(5)

Total number of Same-type trials = 450


,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta
50,C-,1,1,C-,1.0,0.13,0.8,0.8,1.8
307,Money,2,1,Money,4.5,0.38,0.7,0.7,1.8
19,Money,2,1,Money,22.5,0.75,0.7,0.7,1.8
302,Money,2,1,Money,2.0,0.38,0.7,0.7,1.8
444,C-,1,1,C-,11.0,0.75,0.8,0.8,1.8


##### Compute reference and Lottery EUs and calculate pL

In [58]:
def get_EU_(p,X, alpha):
  return p * X**alpha

def get_EU(row, cols):
  p = row[cols[1]]
  X = row[cols[0]]
  alpha = row[cols[2]]
  EU = get_EU_(p,X, alpha)
  return EU

def get_pL_(euL, euR, beta):
  return 1 - 1/(1 + np.exp(beta * (euL - euR)))

def get_pL(row, cols = column_names[8:11]):
  euL = row[cols[2]]
  euR = row[cols[1]]
  beta = row[cols[0]]
  pL = get_pL_(euL, euR, beta)
  return pL

ref_cols = column_names[1:3] + [column_names[6]]
lott_cols = column_names[4:6] + [column_names[7]]

sameType_sim_df[column_names[9]] = sameType_sim_df.apply(lambda row: get_EU(row, ref_cols), axis=1)
sameType_sim_df[column_names[10]] = sameType_sim_df.apply(lambda row: get_EU(row, lott_cols), axis=1)
sameType_sim_df[column_names[11]] = sameType_sim_df.apply(lambda row: get_pL(row), axis=1)
sameType_sim_df.sample(5)

,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta,ref_EU,lott_EU,pL
101,C+,1,1,C+,1.0,0.22,0.8,0.8,1.8,1.0,0.220000,0.197182
63,C-,1,1,C-,5.0,0.50,0.8,0.8,1.8,1.0,1.811949,0.811763
60,C-,1,1,C-,5.0,0.13,0.8,0.8,1.8,1.0,0.471107,0.278481
267,C+,1,1,C+,11.0,0.38,0.8,0.8,1.8,1.0,2.587604,0.945715
402,C+,1,1,C+,1.0,0.38,0.8,0.8,1.8,1.0,0.380000,0.246754


##### Simulate Choices

In [59]:
seed = 1
rng = np.random.default_rng(1)

def get_choice(row, cols = column_names[11]):
  pL = row[cols]
  return rng.binomial(1, pL)

sameType_sim_df[column_names[12]] = sameType_sim_df.apply(lambda row: get_choice(row), axis=1)
sameType_sim_df.sample(5)

,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta,ref_EU,lott_EU,pL,choice
184,C+,1,1,C+,2.0,0.75,0.8,0.8,1.8,1.000000,1.305826,0.634248,1
223,C-,1,1,C-,25.0,0.50,0.8,0.8,1.8,1.000000,6.566320,0.999955,1
304,Money,2,1,Money,2.0,0.75,0.7,0.7,1.8,1.624505,1.218379,0.324969,0
152,Money,2,1,Money,2.0,0.38,0.7,0.7,1.8,1.624505,0.617312,0.140282,0
281,C-,1,1,C-,2.0,0.22,0.8,0.8,1.8,1.000000,0.383042,0.247773,0


#### Estimate Parameters

In [88]:
optimize_cols = column_names[:6]  + [column_names[12]]
def get_likelihood(row, params, cols = optimize_cols):
  ref_type = row[cols[0]]
  lott_type = row[cols[3]]
  if len(params) == 4:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta) = params
    alphas = {'Money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
  elif len(params) == 6:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta_money, beta_Cplus, beta_Cminus) = params
    alphas = {'Money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    betas = {'Money' : beta_money,
             'C+' : beta_Cplus,
             'C-' : beta_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
    beta = betas[lott_type]
  
  choice = row[cols[6]]

  ref_X = row[cols[1]]
  ref_p = row[cols[2]]
  ref_EU = get_EU_(ref_p, ref_X, ref_alpha)
  
  lott_X = row[cols[4]]
  lott_p = row[cols[5]]
  lott_EU = get_EU_(lott_p, lott_X, lott_alpha)

  pL = get_pL_(lott_EU, ref_EU, beta)
  if choice == 1:
    likelihood = pL
  else:
    likelihood = 1 - pL
  return likelihood

def get_negLogLikelihood(params, args):

  df = args
  task_cols = optimize_cols
  # compute likelihood of each choice
  likelihood = df.apply(lambda row: get_likelihood(row, params, cols = task_cols), axis=1).values
  # Take negative of logLikelihood for convention
  negloglikelihood = - np.sum(np.log(likelihood))
  return negloglikelihood


##### 3 alphas + 1 beta

In [90]:
alphaMoney0 = 1
alphaCplus0 = 1
alphaCminus0 = 1
beta0 = 2

args = (sameType_sim_df)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, beta0)
res = minimize(get_negLogLikelihood, x0, args=args )
print('{}\n  - parameters: {}'.format(res.message, res.x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log


Optimization terminated successfully.
  - parameters: [0.71529899 0.8626828  0.84339465 1.85326669]


##### 3 alphas + 3 betas

```
alphaMoney0 = 1
alphaCplus0 = 1
alphaCminus0 = 1
betaMoney0 = 2
betaCplus0 = 2
betaCminus0 = 2

args = (sameType_sim_df)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, betaMoney0, betaCplus0, betaCminus0)
res = minimize(get_negLogLikelihood, x0, args=args )
print('{}\n  - parameters: {}'.format(res.message, res.x))
```

##### Pack results to Dataframe

In [63]:
if len(res.x) == 4:
  alphaEstimators = {'Money' : res.x[0],
             'C+' : res.x[1],
             'C-' : res.x[2]}
  betaEstimators = res.x[3]
  sameType_sim_df[column_names[13]] = sameType_sim_df[column_names[0]].replace(alphaEstimators)
  sameType_sim_df[column_names[14]] = sameType_sim_df[column_names[3]].replace(alphaEstimators)
  sameType_sim_df[column_names[15]] = betaEstimators
elif len(estimatedParameters) == 6:
  alphaEstimators = {'Money' : res.x[0],
             'C+' : res.x[1],
             'C-' : res.x[2]}
  betaEstimators = {'Money' : res.x[3],
             'C+' : res.x[4],
             'C-' : res.x[5]}
  sameType_sim_df[column_names[13]] = sameType_sim_df[column_names[0]].replace(alphaEstimators)
  sameType_sim_df[column_names[14]] = sameType_sim_df[column_names[3]].replace(alphaEstimators)
  sameType_sim_df[column_names[15]] = sameType_sim_df[column_names[3]].replace(betaEstimators)

sameType_sim_df.sample(5)

,ref_type,ref_qt,ref_prob,lott_type,lott_qt,lott_prob,ref_alpha,lott_alpha,beta,ref_EU,lott_EU,pL,choice,ref_alphaEstimates,lott_alphaEstimates,beta_estimates
5,Money,2,1,Money,4.5,0.13,0.7,0.7,1.8,1.624505,0.372556,0.095047,0,0.750359,0.750359,1.402172
179,C+,1,1,C+,1.0,0.75,0.8,0.8,1.8,1.000000,0.750000,0.389361,0,0.823085,0.823085,2.404805
336,C+,1,1,C+,5.0,0.22,0.8,0.8,1.8,1.000000,0.797258,0.409765,1,0.823085,0.823085,2.404805
372,C-,1,1,C-,25.0,0.38,0.8,0.8,1.8,1.000000,4.990403,0.999241,1,0.820708,0.820708,2.153349
158,Money,2,1,Money,4.5,0.50,0.7,0.7,1.8,1.624505,1.432910,0.414627,0,0.750359,0.750359,1.402172


### One Subject

In [74]:
''' 
initGuess:
  (money_alpha0, Cplus_alpha0, Cminus_alpha0, beta0)
                          OR
initGuess: 
  (money_alpha0, Cplus_alpha0, Cminus_alpha0, money_beta0, Cplus_beta0, Cminus_beta0)
'''
def simulate_SameType_Subject(all_refPars, all_lottPars, uniqueComb_reps, alphas, betas, initGuess):
  
  column_names = ['ref_type', 'ref_qt', 'ref_prob' ,
                  'lott_type', 'lott_qt', 'lott_prob',
                  'ref_alpha', 'lott_alpha', 'beta',
                  'ref_EU', 'lott_EU', 'pL', 'choice',
                  'ref_alphaEstimates', 'lott_alphaEstimates', 
                  'beta_estimates']

  # Generate Lotteries 
  LotteryCombinations = get_allCombinations(all_refPars, all_lottPars, uniqueComb_reps)

  # Load Combinations onto DataFrame 
  df = pd.DataFrame(LotteryCombinations, columns = column_names[:6])

  # Get alpha and beta values
  df[column_names[6]] = df[column_names[0]].replace(alphas)
  df[column_names[7]] = df[column_names[3]].replace(alphas)
  df[column_names[8]] = df[column_names[3]].replace(betas)

  # Calculate theoretical EUs and pL
  ref_cols = column_names[1:3] + [column_names[6]]    # ['ref_qt', 'ref_prob', 'ref_alpha']
  lott_cols = column_names[4:6] + [column_names[7]]   # ['lott_qt', 'lott_prob', 'lott_alpha']

  df[column_names[9]] = df.apply(lambda row: get_EU(row, ref_cols), axis=1)
  df[column_names[10]] = df.apply(lambda row: get_EU(row, lott_cols), axis=1)
  df[column_names[11]] = df.apply(lambda row: get_pL(row), axis=1)

  # Simulate Choices based on pL
  df[column_names[12]] = df.apply(lambda row: get_choice(row), axis=1)

  # Parameter Estimation
  optimize_cols = column_names[:6]  + [column_names[12]]  # Goes into loss function
  res = minimize(get_negLogLikelihood, initGuess, args=(df) )
  
  if len(initGuess) == 4:
    alphaEstimators = {'Money' : res.x[0],
              'C+' : res.x[1],
              'C-' : res.x[2]}
    betaEstimators = res.x[3]
    df[column_names[13]] = df[column_names[0]].replace(alphaEstimators)
    df[column_names[14]] = df[column_names[3]].replace(alphaEstimators)
    df[column_names[15]] = betaEstimators 
  elif len(initGuess) == 6:
    alphaEstimators = {'Money' : res.x[0],
              'C+' : res.x[1],
              'C-' : res.x[2]}
    betaEstimators = {'Money' : res.x[3],
              'C+' : res.x[4],
              'C-' : res.x[5]}
    df[column_names[13]] = df[column_names[0]].replace(alphaEstimators)
    df[column_names[14]] = df[column_names[3]].replace(alphaEstimators)
    df[column_names[15]] = df[column_names[3]].replace(betaEstimators)

  return df, res

#### Define parameters and Run simulation

In [75]:
# Reseting a seed to make simulation reproducible
seed = 1
rng = np.random.default_rng(1)

# Task variables

uniqueComb_reps = 6                   # nr. of Ref./Lott. unique comb. repititions  
refPs = [1]                           # probability of Reference choices
lottPs = [0.13, 0.22, 0.38, .50, .75] # probability of Lottery choices

money_id = 'Money'
money_refQs = [2]                     # Euros
money_lottQs = [2, 4.5, 10, 22.5, 50] # Euros

Cplus_id = 'C+'
Cplus_refQs = [1]                     # Cups of C+ yogurt 
Cplus_lottQs = [1, 2, 5, 11, 25]      # Cups of C+ yogurt

Cminus_id = 'C-'
Cminus_refQs = Cplus_refQs            # Cups of C- yogurt 
Cminus_lottQs = Cplus_lottQs          # Cups of C- yogurt 

# %% Subject specific parameters
money_alpha = 0.7                     # money risk aversion parameter
Cplus_alpha = 0.8                     # Cplus risk aversion parameter
Cminus_alpha = 0.8                    # Cminus risk aversion parameter

money_beta = 1.8                      # money noise parameter
Cplus_beta = money_beta               # Cplus noise parameter
Cminus_beta = money_beta              # Cminus noise parameter

initGuess = (1, 1, 1, 2)              # initial guess at subject parameters

''' 
initGuess:
  (money_alpha0, Cplus_alpha0, Cminus_alpha0, beta0)
                          OR
initGuess: 
  (money_alpha0, Cplus_alpha0, Cminus_alpha0, money_beta0, Cplus_beta0, Cminus_beta0)
'''

# Pack Task Parameters
money_refPars = [money_id, money_refQs, refPs]
money_lottPars = [money_id, money_lottQs, lottPs]
Cplus_refPars = [Cplus_id, Cplus_refQs, refPs]
Cplus_lottPars = [Cplus_id, Cplus_lottQs, lottPs]
Cminus_refPars = [Cminus_id, Cminus_refQs, refPs]
Cminus_lottPars = [Cminus_id, Cminus_lottQs, lottPs]

all_refPars = [money_refPars, Cplus_refPars, Cminus_refPars]
all_lottPars = [money_lottPars, Cplus_lottPars, Cminus_lottPars]

# Pack Subject-specific parameters
alphas = {'Money' : money_alpha,
             'C+' : Cplus_alpha,
             'C-' : Cminus_alpha}
betas = {'Money' : money_beta,
             'C+' : Cplus_beta,
             'C-' : Cminus_beta}

# Run simulation
df, res = simulate_SameType_Subject(all_refPars, all_lottPars, uniqueComb_reps, alphas, betas, initGuess)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
